In [1]:
from torch import nn
import torch

import os
from typing import Iterator
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from datasets import load_from_disk, Dataset as HFDataset # Rename to avoid clash

import torch.nn.functional as F

/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import deepspeed

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [1]:
import torch

print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

Allocated: 0.0 MB
Reserved: 0.0 MB


In [4]:
import os, sys, json, importlib
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)

In [66]:
parent

'/home/ubuntu/projects/llm-models/llm-models'

In [33]:
from models.deepseek import model
importlib.reload(model)

from models.deepseek import train
importlib.reload(train)

from models.deepseek import data
importlib.reload(data)

<module 'models.deepseek.data' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/data.py'>

In [6]:
import importlib
importlib.reload(data)

from torch.utils.data import IterableDataset, DataLoader

In [10]:
os.getcwd()

'/home/ubuntu/projects/llm-models/llm-models/notebook'

In [27]:
tinydataset = data.TinyStoriesDataset(data_dir='../../data')

AttributeError: module 'models.deepseek.data' has no attribute 'TinyStoriesDataset'

In [28]:
dataloader = train.DataLoader(tinydataset, batch_size=2)

NameError: name 'tinydataset' is not defined

In [5]:
model_args = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=512, inter_dim=1365, moe_inter_dim=256,
                             kv_lora_rank=128, qk_nope_head_dim=32, qk_rope_head_dim=16, 
                             v_head_dim=32, 
                             n_routed_experts=16, n_layers=4)

In [6]:
model_args = model.ModelArgs(expert_type=model.ExpertType(2),
                             dim=128, inter_dim=341, moe_inter_dim=64,
                             kv_lora_rank=32, qk_nope_head_dim=8, qk_rope_head_dim=4, 
                             v_head_dim=8, 
                             n_routed_experts=16, n_layers=4,
                                original_seq_len=data.CONTEXT_LENGTH)

In [20]:
model_args = model.ModelArgs(expert_type=model.ExpertType(1),
                             dim=128, inter_dim=341, moe_inter_dim=64,
                             kv_lora_rank=32, qk_nope_head_dim=8, qk_rope_head_dim=4, 
                             v_head_dim=8, n_dense_layers=100,
                             n_routed_experts=16, n_layers=4)

In [10]:
training_args = train.TrainingArgs(
    model_args=model_args,
    batch_size=4,
    gradient_accumulation_steps=8,
    max_steps=100000,
    learning_rate=3e-4,
    checkpoint_dir="./checkpoints",
    data_dir="./data"
)

In [11]:
import torch.distributed as dist

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

ValueError: trying to initialize the default process group twice!

In [64]:
trainer = train.Trainer(training_args)

Total parameters: 29,794,944
Trainable parameters: 29,794,944
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...
Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [31]:
trainer.train_step(batch)

loss


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [13]:
for step, batch in enumerate(trainer.train_loader):
    break
    
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [15]:
trainer.device

device(type='cuda', index=0)

In [32]:
import torch._dynamo as dynamo
explanation = dynamo.explain(trainer.model)(x)

In [34]:
# 3. Manually print the findings
print("\n=== Graph Break Report ===")
print(f"Total Graphs: {explanation.graph_count}")
print(f"Total Breaks: {explanation.graph_break_count}")


=== Graph Break Report ===
Total Graphs: 0
Total Breaks: -1


In [33]:
# Print the specific reasons for the breaks
for reason in explanation.break_reasons:
    print(f"Reason: {reason.reason}")
    print(f"Source: {reason.user_stack[0]}")

In [18]:
xx = torch.randint(0, 3000, x.shape)
yy = torch.randint(0, 3000, y.shape)

xx = xx.to(trainer.device, non_blocking=True)
yy = yy.to(trainer.device, non_blocking=True)

In [19]:
from torch._inductor.debug import compile_debug

# Wrap your model/function with compile_debug
compile_debug(trainer.model)(xx)

ImportError: cannot import name 'compile_debug' from 'torch._inductor.debug' (/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_inductor/debug.py)

In [16]:
y = trainer.model(x)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [47]:
logits = trainer.model(x, start_pos=0)

In [51]:
y.view(-1).shape

torch.Size([4096])

In [52]:
logits.view(-1, logits.size(-1)).shape

torch.Size([4096, 102400])

In [77]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0-3): 4 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (head): ColumnParallelLinear()
)

In [14]:
trainer.train()

/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:219: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.args.use_amp, dtype=torch.bfloat16):


Step 10 | Loss: 11.5201 | LR: 1.35e-06 | Tokens/sec: 438974


KeyboardInterrupt: 

In [14]:
trainer.args.gradient_accumulation_steps

8

In [15]:
trainer.global_step

0

In [11]:
trainer.model.compile()

In [12]:
trainer.args.max_steps=100
import time
time.time()
from torch.profiler import profile, ProfilerActivity, record_function


In [13]:
def test_train(backend='inductor', run_profile=False):
    
    print('compiling')
    trainer.model.compile(backend=backend, dynamic=True)
    print('compiled')
    running_loss = 0.0
    start_time = time.time()

    if run_profile:
        print('prof start')
        trainer.prof.start()

    print('prof started')
    for step, batch in enumerate(trainer.train_loader):
        print(step)
        trainer.prof.step()
        if step + 1 > 24:
            break
        # Update learning rate
        lr = trainer.get_lr(trainer.global_step)
        for param_group in trainer.optimizer.param_groups:
            param_group['lr'] = lr
        
        # Training step
        loss = trainer.train_step(batch)
        running_loss += loss
        
        # Update weights after accumulation steps
        if (step + 1) % trainer.args.gradient_accumulation_steps == 0:
            # Gradient clipping
            
            trainer.scaler.unscale_(trainer.optimizer)
            torch.nn.utils.clip_grad_norm_(
                trainer.model.parameters(),
                trainer.args.grad_clip
            )
            
            # Optimizer step
            with record_function("## optimizer ##"):
                trainer.scaler.step(trainer.optimizer)
                trainer.scaler.update()
                trainer.optimizer.zero_grad(set_to_none=True)
            
            trainer.global_step += 1
            trainer.tokens_seen += (
                trainer.args.batch_size * 
                trainer.args.gradient_accumulation_steps * 
                trainer.args.model_args.max_seq_len * 
                trainer.world_size
            )
            
            # Logging
            if trainer.global_step % trainer.args.log_interval == 0 and trainer.is_main_process:
                avg_loss = running_loss / trainer.args.log_interval
                elapsed = time.time() - start_time
                tokens_per_sec = trainer.tokens_seen / elapsed
                
                print(f"Step {trainer.global_step} | "
                      f"Loss: {avg_loss:.4f} | "
                      f"LR: {lr:.2e} | "
                      f"Tokens/sec: {tokens_per_sec:.0f}")
                
                running_loss = 0.0

            if trainer.global_step +1 == 80:
                return

            # Evaluation
            if trainer.global_step % trainer.args.eval_interval == 0:
                val_loss = trainer.evaluate()
                if trainer.is_main_process:
                    print(f"Validation loss: {val_loss:.4f}")
                    
                    if val_loss < trainer.best_val_loss:
                        trainer.best_val_loss = val_loss
                        trainer.save_checkpoint('best')
            
            # Checkpointing
            if trainer.global_step % trainer.args.save_interval == 0 and trainer.is_main_process:
                trainer.save_checkpoint(f'step_{trainer.global_step}')
            
            # Check if training is complete
            if trainer.global_step >= trainer.args.max_steps:
                if trainer.is_main_process:
                    print("Training complete!")
                    trainer.save_checkpoint('final')
                return

    # trainer.prof.stop()


In [18]:
MAX_NUM_OF_MEM_EVENTS_PER_SNAPSHOT: int = 100000

# Start recording memory snapshot history, initialized with a buffer
# capacity of 100,000 memory events, via the `max_entries` field.
torch.cuda.memory._record_memory_history(
   max_entries=MAX_NUM_OF_MEM_EVENTS_PER_SNAPSHOT
)

test_train('inductor')

file_prefix = 'test'
try:
   torch.cuda.memory._dump_snapshot(f"{file_prefix}.pickle")
except Exception as e:
   logger.error(f"Failed to capture memory snapshot {e}")

# Stop recording memory snapshot history.
torch.cuda.memory._record_memory_history(enabled=None)

In [14]:
test_train('inductor', run_profile=True)

compiling
compiled
prof start
prof started
0


[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0] failed while attempting to run meta for aten.add_.Tensor
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0] Traceback (most recent call last):
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0]   File "/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_subclasses/fake_tensor.py", line 2755, in _dispatch_impl
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0]     r = func(*args, **kwargs)
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0]         ^^^^^^^^^^^^^^^^^^^^^
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0]   File "/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/torch/_ops.py", line 841, in __call__
[rank0]:E1228 19:41:52.856000 3149 torch/_subclasses/fake_tensor.py:2759] [0/0]     return self._op(*args, **kwargs)
[rank0]:E1228 1

TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in function iadd>(*(FakeTensor(..., device='cuda:0', size=(4, 1024, 16, 1024),
           grad_fn=<MulBackward0>), FakeTensor(..., size=(4096, 1, 4096))), **{}): got RuntimeError('Attempting to broadcast a dimension of length 4096 at -1! Mismatching argument at index 1 had torch.Size([4096, 1, 4096]); but expected shape should be broadcastable to [4, 1024, 16, 1024]')

from user code:
   File "/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py", line 995, in forward
    h = layer(h, start_pos, freqs_cis, self.mask)
  File "/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py", line 938, in forward
    x = x + self.attn(self.attn_norm(x), start_pos, freqs_cis, mask)
  File "/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py", line 544, in forward
    scores += mask.unsqueeze(1)

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this 

In [24]:
trainer.prof.stop()

In [25]:
trainer.prof.export_chrome_trace("trace.json")

In [26]:
file_prefix = 'memory_profile_1'
trainer.prof.export_memory_timeline(f"{file_prefix}.html", device="cuda:0")

ValueError: with_stack=True required for memory profiling.

In [27]:
print(trainer.prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ## forward ##         0.00%       0.000us         0.00%       0.000us       0.000us        3.781s       232.98%        3.781s     157.526ms           0 B           0 B           0 B           0 

In [29]:
import depyf

In [30]:
depyf.save_code()

AttributeError: module 'depyf' has no attribute 'save_code'

In [17]:
from torch.profiler import profile, ProfilerActivity, record_function

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True,
            profile_memory=True) as prof_inductor:
    test_train('inductor')

print(prof_inductor.key_averages().table(sort_by="self_cuda_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::bmm         3.23%      80.284ms         4.43%     109.930ms      27.264us     362.544ms        23.11%     362.836ms      89.989us           0 B           0 B           0 B           0 

In [13]:
from torch.profiler import profile, ProfilerActivity, record_function

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True,
            profile_memory=True) as prof_inductor:
    test_train('inductor')

print(prof_inductor.key_averages().table(sort_by="self_cuda_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::bmm         1.88%      85.357ms         2.55%     115.821ms      28.605us     362.953ms        23.14%     362.953ms      89.640us           0 B           0 B           0 B           0 

In [12]:
from torch.profiler import profile, ProfilerActivity, record_function

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True,
            profile_memory=True) as prof_eager:
    test_train('eager')

print(prof_eager.key_averages().table(sort_by="self_cuda_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::copy_         3.54%     212.499ms         9.56%     573.137ms      53.479us     471.169ms        26.52%     472.052ms      44.047us           0 B           0 B           0 B           0 

In [19]:
import pandas as pd
avgs = prof_inductor.key_averages()

data_to_print = []
for evt in avgs:
    data_to_print.append(evt.__dict__)
    


In [20]:
import pandas as pd
data_to_print = pd.DataFrame(data_to_print)

In [25]:
data_to_print.columns

Index(['key', 'count', 'node_id', 'is_async', 'is_remote', 'use_device',
       'cpu_time_total', 'device_time_total', 'self_cpu_time_total',
       'self_device_time_total', 'input_shapes', 'overload_name', 'stack',
       'scope', 'cpu_memory_usage', 'device_memory_usage',
       'self_cpu_memory_usage', 'self_device_memory_usage', 'cpu_children',
       'cpu_parent', 'device_type', 'is_legacy', 'flops',
       'is_user_annotation'],
      dtype='object')

In [28]:
data_to_print[['key','self_device_time_total', 'self_device_memory_usage']].sort_values('self_device_memory_usage', ascending=False).head(30).values

array([['aten::empty_strided', 0.0, 61057961984],
       ['aten::nll_loss_backward', 275.1339999999618, 40265318400],
       ['## Call CompiledFxGraph fnoewiyynwx3wzjela47bplx2bny6edpp5rrhvetmp53cdzwa3tj ##',
        0.0, 20413087744],
       ['aten::_log_softmax', 157057.79300000006, 20132659200],
       ['## Call CompiledFxGraph fvingb54prth3cicubokrz6if6r2fwc6l2igx73frkvw3p4ju3om ##',
        0.0, 20132659200],
       ['aten::_log_softmax_backward_data', 162181.84599999932,
        20132659200],
       ['## Call CompiledFxGraph fffrru2g3xvxsfxkqu75k4uerzltzs3m3r5ovkz3qiprlekdbcfx ##',
        0.0, 7028539392],
       ['## Call CompiledFxGraph fe2dw25r4rvtemqmwyujb2x6u35c34covu4sfukjwcvvzkldo3n2 ##',
        0.0, 5593890816],
       ['aten::mul', 12783.020999987377, 2528683520],
       ['aten::index', 9987.224999997532, 1362493440],
       ['aten::empty', 0.0, 1360178688],
       ['## Call CompiledFxGraph f5k6hqiihurixgxoy2de5vpckw5nbbhv5alpsnuext4atlmj42vb ##',
        0.0, 62966988

In [24]:
data_to_print['self_device_memory_usage'].sum()/1e9

np.float64(0.11087872)

In [14]:
print(prof.key_averages().table(sort_by="device_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
       InductorBenchmarker.benchmark_gpu (dynamo_timed)         0.00%       0.000us         0.00%       0.000us       0.000us        3.189s       153.52%        3.189s      18.542ms           0 B           0 B           0 B           0 

In [31]:
data_to_print.sort_values('device_time_total', ascending=False)

,key,count,node_id,is_async,is_remote,use_device,cpu_time_total,device_time_total,self_cpu_time_total,self_device_time_total,...,cpu_memory_usage,device_memory_usage,self_cpu_memory_usage,self_device_memory_usage,cpu_children,cpu_parent,device_type,is_legacy,flops,is_user_annotation
17,aten::copy_,10716,-1,False,False,cuda,564855.953,482483.527,186835.510,482464.744,...,0,0,0,0,[<FunctionEvent id=17 name=Activity Buffer Req...,<FunctionEvent id=15 name=aten::_to_copy overl...,DeviceType.CPU,False,0,False
15,aten::_to_copy,6408,-1,False,False,cuda,584840.836,450528.386,51657.894,0.000,...,11424,150392143872,0,0,[<FunctionEvent id=16 name=aten::empty_strided...,<FunctionEvent id=14 name=aten::to overload_na...,DeviceType.CPU,False,0,False
14,aten::to,8508,-1,False,False,cuda,604355.725,450528.386,19514.889,0.000,...,11424,150392143872,0,0,[<FunctionEvent id=15 name=aten::_to_copy over...,None,DeviceType.CPU,False,0,False
60,aten::bmm,4032,-1,False,False,cuda,148978.121,243491.243,122162.940,243491.243,...,0,45600669696,0,45600669696,[<FunctionEvent id=302 name=cudaDeviceGetAttri...,<FunctionEvent id=108 name=aten::einsum overlo...,DeviceType.CPU,False,0,False
204,autograd::engine::evaluate_function: SoftmaxBa...,600,-1,False,False,cuda,49287.896,242334.867,5418.976,0.000,...,0,-29434576896,0,-58850279424,[<FunctionEvent id=5980 name=SoftmaxBackward0 ...,None,DeviceType.CPU,False,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,aten::slice,1704,-1,False,False,cuda,11913.564,0.000,9207.549,0.000,...,0,0,0,0,[<FunctionEvent id=36 name=aten::as_strided ov...,None,DeviceType.CPU,False,0,False
25,aten::as_strided,47739,-1,False,False,cuda,77996.598,0.000,77996.598,0.000,...,0,0,0,0,[],<FunctionEvent id=29 name=aten::expand overloa...,DeviceType.CPU,False,0,False
1,aten::random_,1,-1,False,False,cuda,21.621,0.000,21.621,0.000,...,0,0,0,0,[],None,DeviceType.CPU,False,0,False
0,aten::empty,13307,-1,False,False,cuda,86342.867,0.000,86342.867,0.000,...,3678076,13019180544,3678076,13019180544,[],None,DeviceType.CPU,False,0,False


In [28]:
from torch._inductor.debug import compile_debug

# Wrap your model/function with compile_debug
compile_debug(your_function)(inputs)

Index(['key', 'count', 'node_id', 'is_async', 'is_remote', 'use_device',
       'cpu_time_total', 'device_time_total', 'self_cpu_time_total',
       'self_device_time_total', 'input_shapes', 'overload_name', 'stack',
       'scope', 'cpu_memory_usage', 'device_memory_usage',
       'self_cpu_memory_usage', 'self_device_memory_usage', 'cpu_children',
       'cpu_parent', 'device_type', 'is_legacy', 'flops',
       'is_user_annotation'],
      dtype='object')

In [31]:
data_to_print.columns

Index(['key', 'count', 'node_id', 'is_async', 'is_remote', 'use_device',
       'cpu_time_total', 'device_time_total', 'self_cpu_time_total',
       'self_device_time_total', 'input_shapes', 'overload_name', 'stack',
       'scope', 'cpu_memory_usage', 'device_memory_usage',
       'self_cpu_memory_usage', 'self_device_memory_usage', 'cpu_children',
       'cpu_parent', 'device_type', 'is_legacy', 'flops',
       'is_user_annotation'],
      dtype='object')

In [75]:
logits = trainer.model(xx, start_pos=0)
loss = F.cross_entropy(
    logits.view(-1, logits.size(-1)),
    yy.view(-1),
    ignore_index=-1
)

In [76]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
[4, 1024, 32]

In [60]:
loss.grad_fn

In [44]:
y.shape

torch.Size([4, 1024, 102400])

In [12]:
for k, v in trainer.model.named_parameters():
    print(v.dtype)
    if v.grad_fn:
        print((k))

torch.float32
torch.bfloat16
torch.bfloat16
torch.float32
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.float32
torch.float32
torch.bfloat16
torch.bfloat16
torch.float32
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.float32
torch.float32
torch.bfloat16
torch.bfloat16
torch.float32
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.float32
torch.float32
torch.bfloat16
torch.bfloat16
torch.float32
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.bfloat16
torch.float32
torch.float32
torch.float32
torch.float32


In [13]:
torch.get_default_dtype()

torch.float32

In [23]:
trainer.model


Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0-3): 4 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (head): ColumnParallelLinear()
)

In [18]:
trainer.train()

/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:218: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.args.use_amp, dtype=torch.bfloat16):


loss
loss


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [9]:
import torch.distributed as dist

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

In [9]:
from dataclasses import dataclass, asdict
import torch.nn.functional as F

In [59]:
count = 0
for step, batch in enumerate(trainer.train_loader):
    count += 1
    if count > 4:
        break
    break

In [50]:
y.shape


torch.Size([4, 1024])

In [51]:
y = torch.randint(0, 10000, y.shape)

In [52]:
x, y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [60]:
logits = trainer.model(x, start_pos=0)

loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss.backward(retain_graph=False)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [57]:
loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

In [55]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [11]:

def enum_to_value(obj):
    if isinstance(obj, Enum):
        return obj.value
    elif isinstance(obj, dict):
        return {k: enum_to_value(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [enum_to_value(v) for v in obj]
    return obj

In [12]:
for step, batch in enumerate(trainer.train_loader):
    print(batch)
    break

[tensor([[ 1263,    13, 40355,  ...,   465,   835,   290],
        [  284, 13502,  2405,  ...,   635, 11040,    13],
        [  356,   460,   526,  ...,   257,  7812,  7838],
        [   11,   644,   460,  ...,   284,  1441,   262]]), tensor([[ 1263,    13, 40355,  ...,   465,   835,   290],
        [  284, 13502,  2405,  ...,   635, 11040,    13],
        [  356,   460,   526,  ...,   257,  7812,  7838],
        [   11,   644,   460,  ...,   284,  1441,   262]])]


In [13]:
x,y = batch
x = x.to(trainer.device, non_blocking=True)
y = y.to(trainer.device, non_blocking=True)

In [14]:
logits.shape

NameError: name 'logits' is not defined

In [34]:
logits1 = trainer.model(x, start_pos=0)

In [35]:
loss1 = F.cross_entropy(
                logits1.view(-1, logits1.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss1.backward()

In [36]:
logits2 = trainer.model(x, start_pos=0)

In [37]:
loss2 = F.cross_entropy(
                logits2.view(-1, logits2.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

loss2.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [22]:
loss1 = F.cross_entropy(
                logits1.view(-1, logits1.size(-1)),
                y.view(-1),
                ignore_index=-1
            )

In [29]:
trainer.model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-3): 3 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (gate): Gate()
        (experts): ModuleList(
          (0-15): 16 x Expert(
            (w1): Linear()
            (w2): Linear()
            (w3): Linear()
          )
        )
        (shared_experts): MLP(
          (w1): ColumnParallelLinear()
          (w2): RowParallelLinear()

In [9]:
trainer = train.Trainer(training_args)

Total parameters: 33,000,576
Trainable parameters: 33,000,576
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...


/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:165: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.args.use_amp)


Tokenized dataset loaded. Total chunks: 459757
Loading *tokenized* TinyStories dataset split from: './processed_tinystories/validation'...
Tokenized dataset loaded. Total chunks: 4619


In [23]:
trainer_model = model.Transformer(trainer.args.model_args)

In [18]:
deepspeed.init_distributed()

In [19]:
trainer_model

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-3): 3 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=128, out_features=16, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-15): 16 x Expert(

In [32]:
trainer.device

device(type='cuda', index=0)

In [33]:
trainer_model.to(trainer.device)

Transformer(
  (embed): ParallelEmbedding()
  (layers): ModuleList(
    (0): Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MLP(
        (w1): ColumnParallelLinear()
        (w2): RowParallelLinear()
        (w3): ColumnParallelLinear()
      )
      (ffn_norm): RMSNorm()
      (attn_norm): RMSNorm()
    )
    (1-3): 3 x Block(
      (attn): MLA(
        (wq): ColumnParallelLinear()
        (wkv_a): Linear()
        (kv_norm): RMSNorm()
        (wkv_b): ColumnParallelLinear()
        (wo): RowParallelLinear()
      )
      (ffn): MoE(
        (experts): MoE(
          (deepspeed_moe): MOELayer(
            (gate): TopKGate(
              (wg): Linear(in_features=128, out_features=16, bias=False)
            )
            (experts): Experts(
              (deepspeed_experts): ModuleList(
                (0-15): 16 x Expert(

In [16]:
trainer_model.mask.device
torch.set_default_device("cuda")

In [33]:
importlib.reload(model)
trainer_model = model.Transformer(trainer.args.model_args)

In [50]:
trainer_model.layers[1].ffn.experts.deepspeed_moe.experts

Experts(
  (deepspeed_experts): ModuleList(
    (0-15): 16 x Expert(
      (block): Block(
        (attn): ExpertMLA(
          (wq): Linear()
          (wkv_a): Linear()
          (kv_norm): RMSNorm()
          (wkv_b): Linear()
          (wo): Linear()
        )
        (attn_norm): RMSNorm()
      )
    )
  )
)

In [79]:
from deepspeed.moe.layer import MOELayer

def get_all_moe_layers(model):
    # This will traverse the entire Transformer/Block/MoE/MOELayer tree
    moe_layers = []
    print(len(list(model.named_modules())))
    for name, module in model.named_modules():
        if isinstance(module, MOELayer):
            moe_layers.append(module)
    return moe_layers

In [82]:
get_all_moe_layers(trainer_model)[0].experts

510


Experts(
  (deepspeed_experts): ModuleList(
    (0-15): 16 x Expert(
      (block): Block(
        (attn): ExpertMLA(
          (wq): Linear()
          (wkv_a): Linear()
          (kv_norm): RMSNorm()
          (wkv_b): Linear()
          (wo): Linear()
        )
        (attn_norm): RMSNorm()
      )
    )
  )
)

In [74]:
list(trainer_model.named_modules())[0]

('',
 Transformer(
   (embed): ParallelEmbedding()
   (layers): ModuleList(
     (0): Block(
       (attn): MLA(
         (wq): ColumnParallelLinear()
         (wkv_a): Linear()
         (kv_norm): RMSNorm()
         (wkv_b): ColumnParallelLinear()
         (wo): RowParallelLinear()
       )
       (ffn): MLP(
         (w1): ColumnParallelLinear()
         (w2): RowParallelLinear()
         (w3): ColumnParallelLinear()
       )
       (ffn_norm): RMSNorm()
       (attn_norm): RMSNorm()
     )
     (1-3): 3 x Block(
       (attn): MLA(
         (wq): ColumnParallelLinear()
         (wkv_a): Linear()
         (kv_norm): RMSNorm()
         (wkv_b): ColumnParallelLinear()
         (wo): RowParallelLinear()
       )
       (ffn): MoE(
         (experts): MoE(
           (deepspeed_moe): MOELayer(
             (gate): TopKGate(
               (wg): Linear(in_features=128, out_features=16, bias=False)
             )
             (experts): Experts(
               (deepspeed_experts): ModuleLi

In [62]:
# trainer.model = trainer_model
# from deepspeed.moe.layer import MOELayer
from model import MoE

ModuleNotFoundError: No module named 'model'

In [67]:
for name, module in trainer.model.named_modules():
    # if isinstance(module, model.MoE):
    if 'MoE' in str(type(module)):
        print(type(module))
        # break


<class 'models.deepseek.model.MoE'>
<class 'deepspeed.moe.layer.MoE'>
<class 'models.deepseek.model.MoE'>
<class 'deepspeed.moe.layer.MoE'>
<class 'models.deepseek.model.MoE'>
<class 'deepspeed.moe.layer.MoE'>


In [58]:
model.MoE

models.deepseek.model.MoE

In [46]:
module.name

AttributeError: 'ColumnParallelLinear' object has no attribute 'name'

In [38]:
trainer.collect_moe_aux_loss()

0.0

In [24]:
logits1 = trainer_model(x, start_pos=0)

torch.Size([4, 1024, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([4, 1024, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([1, 1, 512, 128])
torch.Size([4, 102

In [28]:
for k, v in trainer_model.named_parameters():
    if 'cuda' not in str(v.device):
        print(k)
        print(v.device)
        break

In [10]:
trainer.train()

/home/ubuntu/projects/llm-models/llm-models/models/deepseek/train.py:218: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.args.use_amp, dtype=torch.bfloat16):


loss
loss


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [17]:
for name, param in trainer.model.named_parameters():
    print(param)
    break

AcceleratorError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
id(trainer.optimizer)

273394814022960

In [ ]:
import os
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

# --- Configuration ---
MODEL_NAME = "gpt2"
CONTEXT_LENGTH = 1024
PROCESSED_DATA_DIR = "./processed_tinystories"

def tokenize_and_group_dataset(split: str):
    """Loads a split, tokenizes it, and groups it into fixed-size chunks."""
    
    # 1. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    # 2. Load Raw Dataset
    raw_dataset = load_dataset("roneneldan/TinyStories", split=split)

    # 3. Tokenization
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=False) # No truncation yet
    
    tokenized_datasets = raw_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        num_proc=os.cpu_count(),
    )

    # 4. Grouping and Chunking
    def group_texts(examples):
        # Concatenate all lists of token IDs in the batch
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        
        # Drop the last chunk
        total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
        
        # Split the concatenated list into chunks of CONTEXT_LENGTH
        result = {
            k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
            for k, t in concatenated_examples.items()
        }
        # For CLM, labels are the input IDs
        result["labels"] = result["input_ids"].copy()
        return result

    lm_dataset = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=os.cpu_count(),
    )
    
    # 5. Save the processed dataset
    output_path = os.path.join(PROCESSED_DATA_DIR, split)
    lm_dataset.save_to_disk(output_path)
    print(f"Processed dataset saved to: {output_path}")

    return lm_dataset

# Run this block once to create the processed data files
if not os.path.exists(PROCESSED_DATA_DIR):
    os.makedirs(PROCESSED_DATA_DIR)
    
    print("\n--- Starting Pre-processing for all splits ---")
    tokenize_and_group_dataset("train")
    tokenize_and_group_dataset("validation")
    # TinyStories also has a 'test' split if needed: tokenize_and_group_dataset("test")
    print("--- Pre-processing Complete ---")

/home/ubuntu/projects/llm-models/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL_NAME = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
split = 'train'


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
# 2. Load Raw Dataset
raw_dataset = load_dataset("roneneldan/TinyStories", split=split)

# 3. Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=False) # No truncation yet


In [6]:
tokenized_datasets = raw_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],
        num_proc=os.cpu_count(),
    )

In [1]:
from transformers import AutoTokenizer

MODEL_NAME = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)




/home/ubuntu/projects/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
ds = data.tokenize_and_group_dataset('train')

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-6939c3ad-41a5479f633cd8c13bea7ca3;2614ec0e-0f21-41dd-ae9f-1d4b9b16b6fb)

Entry Not Found for url: https://huggingface.co/api/models/openai-community/gpt2/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [10]:
CONTEXT_LENGTH = 1024

In [11]:
# 4. Grouping and Chunking
def group_texts(examples):
    # Concatenate all lists of token IDs in the batch
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the last chunk
    total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
    
    # Split the concatenated list into chunks of CONTEXT_LENGTH
    result = {
        k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
        for k, t in concatenated_examples.items()
    }
    # For CLM, labels are the input IDs
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=os.cpu_count(),
)

Map (num_proc=4): 100%|██████████████████████████████████████████████████████| 2119719/2119719 [12:34<00:00, 2807.67 examples/s]


In [ ]:
lm_dataset = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=os.cpu_count(),
)

In [8]:
i = 0
for text in raw_dataset:
    print(text)
    batch_data = text
    i += 1
    if i > 3:
        break

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}
{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had man

In [9]:
def group_texts(examples):
    """
    Concatenates all texts and splits them into fixed-size chunks. 
    This is standard practice for CLM training (e.g., training a GPT-2 model).
    """
    # Concatenate all lists of token IDs in the batch
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the last chunk if it's smaller than the context length
    total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH
    
    # Split the concatenated list into chunks of CONTEXT_LENGTH
    result = {
        k: [t[i : i + CONTEXT_LENGTH] for i in range(0, total_length, CONTEXT_LENGTH)]
        for k, t in concatenated_examples.items()
    }
    
    # For Causal Language Modeling (CLM), the 'labels' are the 'input_ids'.
    # The Hugging Face Trainer handles the required shifting for loss calculation.
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
batch_data.keys()

dict_keys(['text'])

In [22]:
class TinyStoriesTokenizedDataset(Dataset):
    """
    A PyTorch Dataset class that loads the *pre-processed*, tokenized, 
    and chunked TinyStories dataset from disk.
    
    It returns a dictionary suitable for a Hugging Face Trainer or custom PyTorch loop.
    """
    def __init__(self, split: str = "train", processed_data_dir: str = "./processed_tinystories"):
        """
        Initializes the dataset by loading the specified split from a local directory.

        Args:
            split (str): The dataset split to load ('train', 'validation').
            processed_data_dir (str): The directory where the pre-processed data is saved.
        """
        file_path = os.path.join(processed_data_dir, split)
        print(f"Loading *tokenized* TinyStories dataset split from: '{file_path}'...")
        
        # Load the pre-processed dataset from disk
        try:
            # We load the processed Arrow Table (Dataset)
            trainer.dataset = lm_dataset
        except Exception as e:
            raise RuntimeError(
                f"Could not load processed dataset split '{split}'. "
                f"Did you run the pre-processing script first? Error: {e}"
            )

        trainer.split = split
        print(f"Tokenized dataset loaded. Total chunks: {len(trainer.dataset)}")

    def __len__(self):
        """
        Returns the total number of fixed-length chunks (examples) in the dataset.
        """
        return len(trainer.dataset)

    def __getitem__(self, idx: int) -> dict:
        """
        Retrieves the token IDs and labels for the chunk at the given index.

        Args:
            idx (int): The index of the chunk to retrieve.

        Returns:
            dict: A dictionary containing 'input_ids' and 'labels' tensors.
        """
        # Retrieve the example from the Hugging Face Dataset
        example = trainer.dataset[idx]
        
        # Convert the lists of integers (token IDs) into PyTorch Tensors
        # These are the necessary keys for Causal Language Modeling training
        return {
            'input_ids': torch.tensor(example['input_ids'], dtype=torch.long),
            'labels': torch.tensor(example['labels'], dtype=torch.long)
        }

In [23]:
train_dataset = TinyStoriesTokenizedDataset('train')

Loading *tokenized* TinyStories dataset split from: './processed_tinystories/train'...
Tokenized dataset loaded. Total chunks: 459751


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler


In [27]:
train_sampler = DistributedSampler(
            train_dataset,
            num_replicas=1,
            rank=0,
            shuffle=True
        )

In [28]:
train_loader = DataLoader(
            train_dataset,
            batch_size=8,
            sampler=train_sampler,
            shuffle=(train_sampler is None),
            num_workers=8,
            pin_memory=True
        )

/home/ubuntu/projects/llm-models/llm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [30]:
for t in train_loader:
    print(t)
    break

{'input_ids': tensor([[20037,  1816,   284,  ...,   640,    11,   612],
        [ 2613,    13,  1119,  ...,   606,   290,   531],
        [  290,   766,   508,  ...,   739,   257, 24484],
        ...,
        [  257,  3704,    13,  ...,   389,   407,  3148],
        [  290,  3835,   290,  ..., 35113,   526,   198],
        [10718,  1306,   284,  ...,    11, 11735,    13]]), 'labels': tensor([[20037,  1816,   284,  ...,   640,    11,   612],
        [ 2613,    13,  1119,  ...,   606,   290,   531],
        [  290,   766,   508,  ...,   739,   257, 24484],
        ...,
        [  257,  3704,    13,  ...,   389,   407,  3148],
        [  290,  3835,   290,  ..., 35113,   526,   198],
        [10718,  1306,   284,  ...,    11, 11735,    13]])}


In [ ]:
b